In [32]:
%%pyspark
df = spark.sql("SELECT * FROM pokerdata.gamedetails")
# df.show(10)

In [33]:
%%pyspark
df.createOrReplaceTempView("vwgamedetails")

In [38]:
spark.sql("DROP TABLE IF EXISTS pokerdata.playerGameSummary")
spark.sql("CREATE TABLE pokerdata.playerGameSummary  (gameId STRING, player string , gameDate string, vpipCt int, pfrCt int, gameCt int, profit float) USING PARQUET")

In [39]:
%%sql
INSERT INTO pokerdata.playerGameSummary
SELECT  t1.gameId
        ,t1.player
       , t1.gameDate
        --, t1.gameTime
        , sum(vpipCt) vpipCt
        , sum(pfrCt) pfrCt
        , sum(gameCt) gameCt
        , sum(profit) profit
FROM (
    SELECT player
        , gameId
        --, CAST(REPLACE(gameTime,' ET','') as time) as gameTime
        , to_date(REPLACE(gameTime,' ET',''),'yyyy/MM/dd') as gameDate
            , MAX(
                    CASE WHEN action = 'bet' THEN 1
                        WHEN action = 'raise' THEN 1
                        WHEN action = 'call' THEN 1
                        ELSE 0 END
                )  AS vpipCt
            , MAX(
                    CASE WHEN action = 'raise' THEN 1 ELSE 0 END 
                ) AS pfrCt
            , 1 AS gameCt
    FROM  vwgameDetails
    WHERE streetName = 'Pre-flop'
            --AND player <> 'Player00008'    
    GROUP BY player
                , gameId
                , gameTime
) t1 JOIN (
    SELECT player, gameId, gameTime, sum(amount) profit
    FROM vwgameDetails
    GROUP BY player, gameId, gameTime
) t2 ON t1.player = t2.player AND t1.gameId = t2.gameId
GROUP BY t1.player, t1.gameId, t1.gameDate--, t1.gameTime

In [22]:
%%sql
select player, sum(profit), sum(vpipCt), sum(pfrCt), sum(gameCt)
from pokerdata.playerGameSummary
group by player
order by 2 desc
LIMIT 10